# Analisis de datos[Valuaciones]

In [153]:
import pandas as pd
import numpy as np
from sklearn.neighbors.kde import KernelDensity
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot
import scipy.stats as st
import statsmodels.datasets
import plotly.plotly as py
from plotly import tools
from datetime import datetime
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from scipy.stats import norm
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
import numpy as np
from scipy import stats as st
%matplotlib inline
import warnings
pd.set_option('precision',2)

Importacíon de datos json

In [168]:
data=pd.read_json(r'C:\Users\luism\OneDrive\Escritorio\data\VALUACION_20190705_174227_TRAYECTORIA_0.json')

In [169]:
diccionario=dict()
for icount in range(len(data.columns)):
    tempkey='resultado_'+str(icount)
    diccionario[icount+1]=data[tempkey]['valuacion']

In [170]:
var=pd.DataFrame.from_dict(diccionario.items())
var=var[1]
var

0    5.66e+07
Name: 1, dtype: float64

In [171]:
# GRÁFICA HISTOGRAMA VALUACIONES
data= [go.Histogram(x=var, nbinsx= 30, name='Valuación')]
layout = go.Layout(title='HISTOGRAMA DE DATOS', xaxis=dict(title=var.name),yaxis=dict(title=''))
fig = go.Figure(data=data,layout=layout)
iplot(fig, filename = 'Hist')

In [172]:
#función de densidad de probabilidad por método paramétrico
def best_fit_distribution(data, bins=200):    
    """Model data by finding best fit distribution to data"""
    '''O. Jaramillo Zuloaga'''
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0
    
    DISTRIBUTIONS = [        
        st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
        st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
        st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
        st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,
        st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,
        st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,
        st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
        st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,
        st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,
        st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy
    ]
    iCount =0    
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf    
    for distribution in DISTRIBUTIONS:        
        try:                
            warnings.filterwarnings('ignore')   
            params = distribution.fit(data)
            arg = params[:-2]
            loc = params[-2]
            scale = params[-1]
            pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
            iCount+=1
            sse = np.sum(np.power(y - pdf, 2.0))                                
            if best_sse > sse:
                best_distribution = distribution
                best_params = params
                best_sse = sse
        except Exception:
            pass 
    return (best_distribution,best_params)


In [173]:
# DISTRIBUCIÓN VALUACIONES - DATOS PESTAÑA 1(VALUACIÓN).
best_dist = best_fit_distribution(var) 

In [174]:
# DISTRIBUCIÓN VALUACIONES
dist = best_dist[0]
args = dist.fit(var)
distxl = best_dist[0].pdf(var, *args)
st.kstest(var, dist.cdf, args)

KstestResult(statistic=0.5945989343842298, pvalue=0.8713525637065429)

In [175]:
# GRÁFICA HISTOGRAMA CON DISTRIBUCION DE PROBABILIDAD JUNTAS
data1 = go.Scatter(x=var,y= (distxl * len(var) * var.max() / 30), mode= 'markers', name= best_dist[0].name)
data2 = go.Histogram(x=var, nbinsx= 30, name='Valuación')
data= [data1, data2]
layout = go.Layout(title='Comparación con distribución obtenida', xaxis=dict(title=var.name),yaxis=dict(title=''))
fig = go.Figure(data=data,layout=layout)
iplot(fig, filename = 'Hist')